In [ ]:
import analysis
import loaders
import json
from utils import yaml_utils
from pathlib import Path

# To work on jupyter
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

# Configurattion section

In [ ]:
# opt comes from the YAML file but not info from there is needed, then we just select what we need :)
opt = yaml_utils.load_yaml("config/data-analysis-demo.yaml", class_mode=True)
db_set = "test" # split set. "train", "test", "original_test","other" # other if implemented.
# db = ["cmu", "h36m", "3dpw", "amass", "expi"] # not validated for CMU in the last pipeline version.
if "cmu" in opt.general_config.data_dir: db = "cmu"
elif "h3" in opt.general_config.data_dir: db = "h36m"
elif "3d" in opt.general_config.data_dir: db = "3dpw"
elif "amas" in opt.general_config.data_dir: db = "amass"
elif "ex" in opt.general_config.data_dir: db = "expi"
dim_used = json.load(open(f'stats/{db}_train_stats.json'))["dim_used"]

# Load dataset and specific set.

In [ ]:
# loader = loaders.get_loader(opt, split="test", model=None, return_all_joints=True, )
loader, actions = loaders.get_loader_divided_by_actions(db, db_set, opt, shuffle=False, return_class=True)

In [ ]:
loader

In [ ]:
curr_act = actions[-2]
target = loader[curr_act].dataset.target[10]
pred = loader[curr_act].dataset.target[75]
print(pred.shape, target.shape)

# Visualization of specific pose sequences given numpy

In [ ]:
args = opt.evaluation_config.visualization.__dict__
args["db"] = db
# {'repeat': 2, 'mode': 'single', 'plot_joints': True, 'db': 'expi'}
analysis.create_animation(f'test_sample{0}.gif',
                          [target],
                          **args)
analysis.create_animation(f'test_sample{1}.gif',
                          [pred],
                          **args)
plt.close()

In [ ]:
data = loader[curr_act].dataset.target
print(data.shape)
plot = analysis.analysis_utils.SequenceAnalytics(data,
                                                 db=db,
                                                 dim_used=dim_used,
                                                 remove_temporal_data=True)

In [ ]:
global_config = {'module': 'norm'}
fig_size = [18, 18]
valid_j = ['R_Wrist_End', 'L_Wrist_End', 'LeftFoot', 'RightFoot', 'Head']
plot_type = {'absolute': {
                            'pose': None, # plot pose with absolute values
                            'velocity': {'ylim': [-1, 50]}, # plot velocity with absolute values
                            'acceleration': {'ylim': [-1, 100]}, # plot acceleration with absolute values
                            'angle': {'ylim': [-1, 1.25]} # plot angle with absolute values
                        },
             'relative': {
                            'pose': None, # plot pose with respect to the pose 0.
                            'velocity': {'ylim': [-1, 600]}, # plot velocity with respect to the velocity of pose 0.
                            'acceleration': {'ylim': [-1, 600]}, # plot acceleration with respect to the acceleration of pose 0.
                            'angle': {'ylim': [-0.01, 0.25]} # plot angle with respect to the angle of pose 0.
                        },
             }

plot.init_figure(size=fig_size)
plot.Plot2D_joint_physics(eval_physical_config=conf, global_config=global_config, mode=name, joints=valid_j) # idx is optional with one sample.
plot.show(f'one_sample.jpg', show=False)
plt.close("all")

for idx in range(5): # number of indexes otherwise 0 is ok.
    for name, conf in plot_type.items():
        plot.init_figure(size=fig_size)
        plot.Plot2D_joint_physics(eval_physical_config=conf, idx=idx, global_config=global_config, mode=name, joints=valid_j)
        plot.show(f'{name}_{idx}.jpg', show=False)
        plt.close("all")

# Data Analytics inside the pipeline (still on validation on YAML)

In [ ]:
plot = analysis.analysis_utils.SequenceAnalytics(loader[actions[0]],
                                                 db=db,
                                                 dim_used=dim_used,
                                                 remove_temporal_data=True)

In [ ]:
for typ in opt.evaluation_config.sets:
    db_set = list(typ.__dict__.keys())[0]  # must have only one set.
    typ = typ.__dict__[db_set]
    actions = "all" if typ.classes == ["all"] else typ.classes
    valid_j = typ.joints  # joint names to analyze.
    indices_to_eval = typ.index  # samples to analyze - Of course they change the order when different set are read
    figs_path = "test.png"
    for idx in indices_to_eval:
        plot_type = typ.evaluate.index.physical.__dict__
        if hasattr(typ.evaluate.index.physical, "global_config"):
            global_config = typ.evaluate.index.physical.global_config
            plot_type.pop("global_config")
        if hasattr(typ.evaluate.index.physical, "fig_size"):
            fig_size = typ.evaluate.index.physical.fig_size
            plot_type.pop("fig_size")
        for name, conf in plot_type.items():
            plot.init_figure(size=fig_size)
            plot.Plot2D_joint_physics(eval_physical_config=conf, idx=idx, global_config=global_config, mode=name, joints=valid_j)
            plot.show(f'pipeline__{name}_{idx}.jpg', show=False)
            plt.close("all")